# Optuna

In [1]:
import pandas as pd
df_test = pd.read_csv("../data/raw/TestingData.csv")
df_train = pd.read_csv("../data/raw/HistoricalTrainingData.csv")
df_gradient = pd.read_csv("../data/processed/customer_sensitivity_metrics.csv")

In [2]:
import sys
sys.path.append('C:/projects/python/coupon-optimisation')
from src.preprocessing import preprocess_and_split

X_train, X_test, y_train, y_test, features = preprocess_and_split(df_train, df_test, df_gradient)

features = [
            'CouponUsageRate',
            'TotalOfferAmtRedeemed',
            'TotalVisits', 
            'TotalSpendAmt',
            'TotalCouponRedeem', 
            'OfferAmt', 
            'AverageOfferAmtPerVisit',
            'AverageSpendPerVisit',
            'TotalOfferAmtReceived',
            'n-1_spend',
            'n-2_spend',
            'n-3_spend',
            'n-4_spend',
            'n-5_spend',
            'MaxSpend',
            'sensitivity_gradient', 
            'MinOfferAmt', 
            'MaxOfferAmt',
            'MedianSpendPerOfferAmt',
            'Year',
            'PatronID',
            'Month',
            'weekNum'
           ] 


C:\projects/python/coupon-optimisation\src\preprocessing\preprocessing.py:60: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  agg_features['MedianSpendPerOfferAmt'].replace([np.inf, -np.inf], 1, inplace=True)


In [3]:
from src.models.GradientBoostClassifier import GBMTrainer

In [4]:
import optuna
import numpy as np
from sklearn.metrics import f1_score

def objective(trial):
    # Suggest a subset of features. We generate boolean flags for each feature.
    selected_features = [trial.suggest_categorical(f'feature_{i}', [True, False]) for i in range(X_train.shape[1])]
    
    # Filter the features for both training and testing based on the selection
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]
    
    # Initialize and train the model on the selected features
    trainer = GBMTrainer()
    trainer.train(X_train_selected, y_train)
    
    # Predict on the test set
    predictions = trainer.predict(X_test_selected)
    predicted_labels = (predictions > 0.5).astype(int)
    
    # Evaluate the model using F1 score
    f1 = f1_score(y_test, predicted_labels)
    
    return f1



C:\Users\jtren\anaconda3\envs\bayes-optimisation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=250)

# Output the best trial results
print(f'Best F1 Score: {study.best_value}')
best_features = [feature for feature, use in zip(features, study.best_params.values()) if use]
print(f'Best features: {best_features}')


[I 2024-04-10 14:25:22,984] A new study created in memory with name: no-name-ac60c896-b4a2-4007-864b-1b006caef79c
[I 2024-04-10 14:25:32,890] Trial 0 finished with value: 0.6476703174308521 and parameters: {'feature_0': True, 'feature_1': False, 'feature_2': True, 'feature_3': False, 'feature_4': False, 'feature_5': False, 'feature_6': False, 'feature_7': True, 'feature_8': True, 'feature_9': True, 'feature_10': False, 'feature_11': False, 'feature_12': False, 'feature_13': False, 'feature_14': False, 'feature_15': True, 'feature_16': True, 'feature_17': False, 'feature_18': False, 'feature_19': True, 'feature_20': True, 'feature_21': True, 'feature_22': False}. Best is trial 0 with value: 0.6476703174308521.


Model trained.


[I 2024-04-10 14:25:42,810] Trial 1 finished with value: 0.6477060798209623 and parameters: {'feature_0': True, 'feature_1': True, 'feature_2': False, 'feature_3': False, 'feature_4': True, 'feature_5': False, 'feature_6': False, 'feature_7': False, 'feature_8': True, 'feature_9': False, 'feature_10': True, 'feature_11': False, 'feature_12': False, 'feature_13': False, 'feature_14': False, 'feature_15': False, 'feature_16': True, 'feature_17': True, 'feature_18': True, 'feature_19': True, 'feature_20': False, 'feature_21': True, 'feature_22': False}. Best is trial 1 with value: 0.6477060798209623.


Model trained.


[I 2024-04-10 14:25:59,204] Trial 2 finished with value: 0.6980328166077417 and parameters: {'feature_0': True, 'feature_1': False, 'feature_2': True, 'feature_3': False, 'feature_4': False, 'feature_5': True, 'feature_6': True, 'feature_7': False, 'feature_8': True, 'feature_9': True, 'feature_10': True, 'feature_11': True, 'feature_12': False, 'feature_13': True, 'feature_14': True, 'feature_15': True, 'feature_16': False, 'feature_17': True, 'feature_18': False, 'feature_19': True, 'feature_20': True, 'feature_21': True, 'feature_22': True}. Best is trial 2 with value: 0.6980328166077417.


...

Model trained.


[I 2024-04-10 15:18:34,873] Trial 248 finished with value: 0.7130003672420125 and parameters: {'feature_0': False, 'feature_1': True, 'feature_2': False, 'feature_3': False, 'feature_4': True, 'feature_5': True, 'feature_6': True, 'feature_7': True, 'feature_8': True, 'feature_9': True, 'feature_10': False, 'feature_11': True, 'feature_12': True, 'feature_13': False, 'feature_14': False, 'feature_15': False, 'feature_16': True, 'feature_17': False, 'feature_18': False, 'feature_19': True, 'feature_20': False, 'feature_21': True, 'feature_22': True}. Best is trial 36 with value: 0.7130003672420125.


Model trained.


[I 2024-04-10 15:18:49,124] Trial 249 finished with value: 0.7040702016430173 and parameters: {'feature_0': False, 'feature_1': True, 'feature_2': False, 'feature_3': False, 'feature_4': True, 'feature_5': True, 'feature_6': False, 'feature_7': True, 'feature_8': True, 'feature_9': True, 'feature_10': False, 'feature_11': True, 'feature_12': True, 'feature_13': False, 'feature_14': False, 'feature_15': False, 'feature_16': True, 'feature_17': False, 'feature_18': False, 'feature_19': True, 'feature_20': False, 'feature_21': True, 'feature_22': True}. Best is trial 36 with value: 0.7130003672420125.


Model trained.
Best F1 Score: 0.7130003672420125
Best features: ['TotalOfferAmtRedeemed', 'TotalCouponRedeem', 'OfferAmt', 'AverageOfferAmtPerVisit', 'AverageSpendPerVisit', 'TotalOfferAmtReceived', 'n-1_spend', 'n-3_spend', 'n-4_spend', 'MinOfferAmt', 'Year', 'Month', 'weekNum']
